### balance_functions

Functions that were made during part 3 to calculate profit and keep track of balances across various bookmakers accounts.

In [17]:
import pandas as pd

In [18]:
bet_spreadsheet = pd.read_csv('Bet Spreadsheet with missing bets added.csv')

bet_spreadsheet.head(5)

,Date Created,Sport,Event,Event Time,Bookie,Bet Type,Type,Outcome,Stake,Odds,Fee (%),Liability,Return,Potential Profit,Bet Result,Note,Profit ID
0,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Betfred,Qualifying,Back,Leicester,10.00,2.05,0,0.00,10.50,-0.43,Lose,,7967100
1,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Smarkets,Qualifying,Lay,Leicester,9.76,2.12,2,10.93,9.56,-0.44,Win,,7967100
2,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Coral,Qualifying,Back,Rochdale,5.00,1.91,0,0.00,4.55,-0.10,Win,,7967200
3,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Smarkets,Qualifying,Lay,Rochdale,5.00,1.93,2,4.65,4.90,-0.10,Lose,,7967200
4,19/11/2019 14:39:18,Football,Wales v Hungary,19/11/2019 19:45:00,Coral,Free (SNR),Back,Hungary,20.00,6.50,0,0.00,110.00,15.92,Lose,,7967346


Functions to caluclate profit

In [19]:
def add_profit_column(spreadsheet):
    """Creates a profit column to the right of bet result.
    
       All values are initialised to 0.00."""
     
    column_after_bet_result = spreadsheet.columns.get_loc('Bet Result') + 1
    spreadsheet.insert(column_after_bet_result, 'Profit', 0.00)
    
    return spreadsheet

bet_spreadsheet = add_profit_column(bet_spreadsheet)

bet_spreadsheet.head(5)

,Date Created,Sport,Event,Event Time,Bookie,Bet Type,Type,Outcome,Stake,Odds,Fee (%),Liability,Return,Potential Profit,Bet Result,Profit,Note,Profit ID
0,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Betfred,Qualifying,Back,Leicester,10.00,2.05,0,0.00,10.50,-0.43,Lose,0.0,,7967100
1,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Smarkets,Qualifying,Lay,Leicester,9.76,2.12,2,10.93,9.56,-0.44,Win,0.0,,7967100
2,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Coral,Qualifying,Back,Rochdale,5.00,1.91,0,0.00,4.55,-0.10,Win,0.0,,7967200
3,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Smarkets,Qualifying,Lay,Rochdale,5.00,1.93,2,4.65,4.90,-0.10,Lose,0.0,,7967200
4,19/11/2019 14:39:18,Football,Wales v Hungary,19/11/2019 19:45:00,Coral,Free (SNR),Back,Hungary,20.00,6.50,0,0.00,110.00,15.92,Lose,0.0,,7967346


In [20]:
def calculate_profit(spreadsheet):
    """Calculates profit based on bet results."""
    
    win_lose_boolean = (spreadsheet['Bet Result'] == 'Win')
    bet_profit = spreadsheet['Potential Profit'] * win_lose_boolean
    spreadsheet.loc[:, 'Profit'] = bet_profit
    
    return spreadsheet

bet_spreadsheet = calculate_profit(bet_spreadsheet)

bet_spreadsheet

,Date Created,Sport,Event,Event Time,Bookie,Bet Type,Type,Outcome,Stake,Odds,Fee (%),Liability,Return,Potential Profit,Bet Result,Profit,Note,Profit ID
0,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Betfred,Qualifying,Back,Leicester,10.00,2.05,0,0.00,10.50,-0.43,Lose,-0.00,,7967100
1,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Smarkets,Qualifying,Lay,Leicester,9.76,2.12,2,10.93,9.56,-0.44,Win,-0.44,,7967100
2,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Coral,Qualifying,Back,Rochdale,5.00,1.91,0,0.00,4.55,-0.10,Win,-0.10,,7967200
3,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Smarkets,Qualifying,Lay,Rochdale,5.00,1.93,2,4.65,4.90,-0.10,Lose,-0.00,,7967200
4,19/11/2019 14:39:18,Football,Wales v Hungary,19/11/2019 19:45:00,Coral,Free (SNR),Back,Hungary,20.00,6.50,0,0.00,110.00,15.92,Lose,0.00,,7967346
5,19/11/2019 14:39:18,Football,Wales v Hungary,19/11/2019 19:45:00,Smarkets,Free (SNR),Lay,Hungary,16.22,6.80,2,94.08,15.90,15.90,Win,15.90,,7967346
6,19/11/2019 14:56:02,Football,North Macedonia v Israel,19/11/2019 19:45:00,Skybet,Qualifying,Back,North Macedonia,5.00,2.40,0,0.00,7.00,-0.33,Win,-0.33,,7967454
7,19/11/2019 14:56:02,Football,North Macedonia v Israel,19/11/2019 19:45:00,Smarkets,Qualifying,Lay,North Macedonia,4.76,2.54,2,7.33,4.66,-0.34,Lose,-0.00,,7967454
8,20/11/2019 14:13:10,Football,Lincoln v Ipswich,20/11/2019 19:45:00,Skybet,Free (SNR),Back,1-2,20.00,11.00,0,0.00,200.00,15.08,Lose,0.00,,7975857
9,20/11/2019 14:13:10,Football,Lincoln v Ipswich,20/11/2019 19:45:00,Smarkets,Free (SNR),Lay,1-2,15.41,13.00,2,184.92,15.10,15.10,Win,15.10,,7975857


In [21]:
def remove_negative_zeros(spreadsheet):
    """Removes the -0.00 from profit column."""
    
    remove_minus = lambda profit : abs(profit) if (profit == -0.00) else profit
    spreadsheet.loc[:, 'Profit'] = spreadsheet.loc[:, 'Profit'].apply(remove_minus)
    
    return spreadsheet

bet_spreadsheet = remove_negative_zeros(bet_spreadsheet)

bet_spreadsheet.head(5)

,Date Created,Sport,Event,Event Time,Bookie,Bet Type,Type,Outcome,Stake,Odds,Fee (%),Liability,Return,Potential Profit,Bet Result,Profit,Note,Profit ID
0,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Betfred,Qualifying,Back,Leicester,10.00,2.05,0,0.00,10.50,-0.43,Lose,0.00,,7967100
1,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Smarkets,Qualifying,Lay,Leicester,9.76,2.12,2,10.93,9.56,-0.44,Win,-0.44,,7967100
2,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Coral,Qualifying,Back,Rochdale,5.00,1.91,0,0.00,4.55,-0.10,Win,-0.10,,7967200
3,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Smarkets,Qualifying,Lay,Rochdale,5.00,1.93,2,4.65,4.90,-0.10,Lose,0.00,,7967200
4,19/11/2019 14:39:18,Football,Wales v Hungary,19/11/2019 19:45:00,Coral,Free (SNR),Back,Hungary,20.00,6.50,0,0.00,110.00,15.92,Lose,0.00,,7967346


In [22]:
def running_profit(spreadsheet):
    """Adds a running profit column to the right of profit."""
    
    column_after_profit = spreadsheet.columns.get_loc('Profit') + 1
    spreadsheet.insert(column_after_profit, 'Running Profit', 
                           spreadsheet['Profit'].cumsum())
    
    return spreadsheet

bet_spreadsheet = running_profit(bet_spreadsheet)

bet_spreadsheet.head(10)

,Date Created,Sport,Event,Event Time,Bookie,Bet Type,Type,Outcome,Stake,Odds,Fee (%),Liability,Return,Potential Profit,Bet Result,Profit,Running Profit,Note,Profit ID
0,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Betfred,Qualifying,Back,Leicester,10.00,2.05,0,0.00,10.50,-0.43,Lose,0.00,0.00,,7967100
1,19/11/2019 13:48:04,Football,Brighton and Hove Albion v Leicester,23/11/2019 15:00:00,Smarkets,Qualifying,Lay,Leicester,9.76,2.12,2,10.93,9.56,-0.44,Win,-0.44,-0.44,,7967100
2,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Coral,Qualifying,Back,Rochdale,5.00,1.91,0,0.00,4.55,-0.10,Win,-0.10,-0.54,,7967200
3,19/11/2019 14:27:29,Football,Rochdale v Wrexham,19/11/2019 19:45:00,Smarkets,Qualifying,Lay,Rochdale,5.00,1.93,2,4.65,4.90,-0.10,Lose,0.00,-0.54,,7967200
4,19/11/2019 14:39:18,Football,Wales v Hungary,19/11/2019 19:45:00,Coral,Free (SNR),Back,Hungary,20.00,6.50,0,0.00,110.00,15.92,Lose,0.00,-0.54,,7967346
5,19/11/2019 14:39:18,Football,Wales v Hungary,19/11/2019 19:45:00,Smarkets,Free (SNR),Lay,Hungary,16.22,6.80,2,94.08,15.90,15.90,Win,15.90,15.36,,7967346
6,19/11/2019 14:56:02,Football,North Macedonia v Israel,19/11/2019 19:45:00,Skybet,Qualifying,Back,North Macedonia,5.00,2.40,0,0.00,7.00,-0.33,Win,-0.33,15.03,,7967454
7,19/11/2019 14:56:02,Football,North Macedonia v Israel,19/11/2019 19:45:00,Smarkets,Qualifying,Lay,North Macedonia,4.76,2.54,2,7.33,4.66,-0.34,Lose,0.00,15.03,,7967454
8,20/11/2019 14:13:10,Football,Lincoln v Ipswich,20/11/2019 19:45:00,Skybet,Free (SNR),Back,1-2,20.00,11.00,0,0.00,200.00,15.08,Lose,0.00,15.03,,7975857
9,20/11/2019 14:13:10,Football,Lincoln v Ipswich,20/11/2019 19:45:00,Smarkets,Free (SNR),Lay,1-2,15.41,13.00,2,184.92,15.10,15.10,Win,15.10,30.13,,7975857


Functions to calculate balances for each bookmaker.

This was my original way of calculating the balances of each bookmaker for the qualifying bets.

The process of creating bookie_index_list was generalised and became the value_index_list() function, which may be useful in the future.

In [23]:
winning_qualifying_bets = (bet_spreadsheet['Bet Type'] == 'Qualifying')   \
                     & (bet_spreadsheet['Bet Result'] == 'Win')

bookie_index_list = {}
for index, bookie in bet_spreadsheet[winning_qualifying_bets]['Bookie'].iteritems():
    bookie_index_list.setdefault(bookie, []).append(index)
        
for bookie, index_list in bookie_index_list.items():
    qualifying_bets_balance = sum(bet_spreadsheet.loc[index_list, 'Stake']   \
                            + bet_spreadsheet.loc[index_list, 'Return'])
    print("{} {}".format(bookie, qualifying_bets_balance))

Smarkets 19.32
Coral 9.55
Skybet 12.0
William Hill 18.5
BetWay 56.1


Using .groupby('Bookie') to calculate the balances turns out to be less complicated.

In [24]:
def qualifying_bet_balances(spreadsheet):
    """Returns a series with the balance for each bookie due to all 
       settled qualifying bets."""
    
    winning_qualifying_bets = (spreadsheet['Bet Type'] == 'Qualifying')   \
                               & (spreadsheet['Bet Result'] == 'Win')
    
    qualifying_by_bookie = spreadsheet[winning_qualifying_bets].groupby('Bookie') 
    
    # Our original stake is returned with qualifying bets
    qualifying_balances = qualifying_by_bookie['Stake'].sum()   \
                             + qualifying_by_bookie['Return'].sum()
    
    # Smarkets balance will be calculated separately 
    if 'Smarkets' in qualifying_balances.index:
        qualifying_balances.drop('Smarkets', inplace=True)
        
    return qualifying_balances

qualifying_bet_balances(bet_spreadsheet)

Bookie
BetWay          56.10
Coral            9.55
Skybet          12.00
William Hill    18.50
dtype: float64

In [25]:
def free_bet_balances(spreadsheet):
    """Returns a series with the balance for each bookie due to all 
       settled free bets."""

    winning_free_bets = (spreadsheet['Bet Type'] == 'Free (SNR)') \
                               & (spreadsheet['Bet Result'] == 'Win')
    
    free_by_bookie = spreadsheet[winning_free_bets].groupby('Bookie')
    
    # Our original state is not returned with free bets
    # SNR means "stake not returned"
    free_balances = free_by_bookie['Return'].sum()
    
    # Smarkets balance will be calculated separately 
    if 'Smarkets' in free_balances.index:
        free_balances.drop('Smarkets', inplace=True)
    
    return free_balances

free_bet_balances(bet_spreadsheet)

Series([], Name: Return, dtype: float64)

In [26]:
qualifying_bet_balances(bet_spreadsheet) + free_bet_balances(bet_spreadsheet)


Bookie
BetWay         NaN
Coral          NaN
Skybet         NaN
William Hill   NaN
dtype: float64

Add series could be useful in future which is way the syntax is generalised.

In [27]:
def add_series(series_1, series_2):
    """Removes NaNs when two series are added together and some indices don't match.
       Indices which don't match keep values from original series.
       Still works if all indices from both series match."""
    
    potential_sum = series_1 + series_2
    
    removed_NaN_sum = potential_sum.fillna(series_1)  \
                                   .fillna(series_2)
    return removed_NaN_sum

add_series(qualifying_bet_balances(bet_spreadsheet), free_bet_balances(bet_spreadsheet))

Bookie
BetWay          56.10
Coral            9.55
Skybet          12.00
William Hill    18.50
dtype: float64

In [28]:
def bookie_balances(spreadsheet):
    """Returns the total balance for each bookmaker for all settled bets."""
    
    return add_series(qualifying_bet_balances(spreadsheet), free_bet_balances(spreadsheet))

bookie_balances(bet_spreadsheet)

Bookie
BetWay          56.10
Coral            9.55
Skybet          12.00
William Hill    18.50
dtype: float64

The generalised version of finding bookie_index_list from before

In [29]:
def value_index_list(series):
    """Iterates through values in a pandas series and returns a dictionary of the form
       {value : [indices for value]}

       Originally used to calculate qualifying bet balances before I realised that it was
       easier to use groupby instead.

       Could be useful in the future."""

    value_indices_dictionary = {}

    for index, value in series.iteritems():
        value_indices_dictionary.setdefault(value, []).append(index)

    return value_indices_dictionary

value_index_list(bet_spreadsheet['Bookie'])

{'Betfred': [0, 18],
 'Smarkets': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
 'Coral': [2, 4],
 'Skybet': [6, 8],
 'William Hill': [10, 12],
 'BetWay': [14, 16]}